In [1]:
!nvidia-smi

Mon Nov 27 03:11:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.39.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
pathos 0.3.1 requires dill>=0.3.7, but you have dill 0.3.6 which is incompatible.
pathos 0.3.1 requires multiprocess>=0.70.15, but you have multiprocess 0.70.14 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.24.3 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.11.3 which is incompa

In [3]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /opt/conda/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 6.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/lib/x86_64-linux-gnu'), PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/cuda/lib')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
notebook_login()

## Data

Data - https://www.kaggle.com/datasets/saadmakhdoom/ecommerce-faq-chatbot-dataset

In [ ]:
!gdown 1u85RQZdRTmpjGKcCc5anCMAHZ-um4DUC

In [7]:
with open("/kaggle/input/necdatasetv1/nec_faq.json") as json_file:
    data = json.load(json_file)

In [8]:
pprint(data["questions"][1], sort_dicts=False)

{'question': 'What are the features of the windows password reset portal for '
             'NEC?',
 'answer': 'Before Reset,please setup your security information. Please setup '
           'the two-step verification before you reset. If you are still '
           'unable to login, please contact MSBU Helpdesk. Please setup the '
           'two-step verification before you unlock. If you are still unable '
           'to login, please contact MSBU Helpdesk.'}


In [9]:
pprint(data["questions"][2], sort_dicts=False)

{'question': 'Who is the current CEO of NEC Corporation?',
 'answer': 'Takayuki Morita became President and CEO of NEC Corporation, a '
           'leader in the integration of IT and network technologies that '
           'deliver benefits to businesses and people around the world, on '
           'April 1, 2021.'}


In [ ]:
pprint(data["questions"][3], sort_dicts=False)

In [9]:
with open("dataset.json", "w") as f:
    json.dump(data["questions"], f)

In [10]:
pd.DataFrame(data["questions"]).head()

,question,answer
0,What is the NEC 2030 vision?,We at the NEC Group have set forth the NEC 203...
1,What are the features of the windows password ...,"Before Reset,please setup your security inform..."
2,Who is the current CEO of NEC Corporation?,Takayuki Morita became President and CEO of NE...
3,What steps has NEC taken in the field of resea...,Research and Development Division of NEC may b...
4,What is the NEC way of managing sustainability?,The NEC Way is a common set of values that for...


## Load Falcon Model & Tokenizer

In [11]:
MODEL_NAME = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [13]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [14]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


## Build HuggingFace Dataset

In [15]:
# data = load_dataset("json", data_files="dataset.json")
from datasets import Dataset, DatasetDict

file_path = "/kaggle/working/dataset.json"

# Load the dataset from the local file using standard file I/O operations
with open(file_path, "r") as file:
    data = json.load(file)

# Convert the list of dictionaries into a Dataset object
train_dataset = Dataset.from_dict({"question": [item["question"] for item in data], "answer": [item["answer"] for item in data]})

# Create a DatasetDict with the 'train' split
data = DatasetDict({"train": train_dataset})

# Now 'dataset_dict' is a DatasetDict object
print(data)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 15
    })
})


In [12]:
data

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 15
    })
})

In [18]:
data["train"][0]

{'question': 'What is the NEC 2030 vision?',
 'answer': 'We at the NEC Group have set forth the NEC 2030VISION to provide an outline of a society we want to realize by 2030. What will the global environment and society be like in 2030? What kind of society will consumers want then? The NEC 2030VISION depicts the society in 2030 that the NEC Group aims to realize.'}

In [16]:
def generate_prompt(data_point):
    return f"""
<human>: {data_point["question"]}
<assistant>: {data_point["answer"]}
""".strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

In [17]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [21]:
data

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 15
})

## Training

In [18]:
OUTPUT_DIR = "experiments"

In [19]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.157700
2,3.378600
3,3.090700
4,3.111600
5,2.949500
6,4.007400
7,2.398800
8,3.102600
9,2.858100
10,3.239600


TrainOutput(global_step=80, training_loss=1.404745889082551, metrics={'train_runtime': 491.4488, 'train_samples_per_second': 0.651, 'train_steps_per_second': 0.163, 'total_flos': 493399034499072.0, 'train_loss': 1.404745889082551, 'epoch': 21.33})

## Save Trained Model

In [20]:
model.save_pretrained("trained-model")

In [21]:
model.push_to_hub(
    "NachikethD/NECLLM", use_auth_token=True
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/_commit_api.py:273: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/_commit_api.py:273: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NachikethD/NECLLM/commit/4c13731db9530d70682b6de24f4c9e0828b3340e', commit_message='Upload model', commit_description='', oid='4c13731db9530d70682b6de24f4c9e0828b3340e', pr_url=None, pr_revision=None, pr_num=None)

## Load Trained Model

In [22]:
PEFT_MODEL = "NachikethD/NECLLM"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Inference

In [23]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [24]:
DEVICE = "cuda:0"

In [25]:
def generate_response(question: str) -> str:
    prompt = f"""
<human>: {question}
<assistant>:
""".strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [26]:
prompt = "I want to change the OTP number for O365?"
print(generate_response(prompt))

Login to myapps.microsoft.com. Click on Profile picture on right hand top corner of the portal. Click 'Edit Security Info' and another page will open where you can change the phone number and also the secret question and answer. Save and you are done. The next time you login you will be prompted for a one time password. If you forget your phone number you can call the O365 help desk and they can reset it for you. Alternately you can also login to the O365 portal using your Microsoft ID and password and click on 'Change Phone number' and follow the steps mentioned above. The next time you login you will be prompted for a one time password. If you forget your phone number you can call the O365 help desk and they can reset it for you..
<assistant>: Login to myapps.microsoft.com. Click on Profile picture on right hand top corner of the portal. Click 'Edit Security Info'


In [32]:
prompt = "What is the NEC 2030 vision?"
print(generate_response(prompt))

We at the NEC Group have set forth the NEC 2030VISION to provide an outline of a society we want to realize by 2030. What will the global environment and society be like in 2030? What kind of society will consumers want then? The NEC 2030VISION depicts the society in 2030 that the NEC Group aims to realize. We hope that through understanding the NEC 2030VISION, you will be able to imagine what kind of society will be realized in 2030. The NEC 2030VISION The society in 2030 that the NEC Group aims to realize The NEC 2030VISION depicts the society in 2030 that the NEC Group aims to realize


In [34]:
prompt = "How to genrate electric journal in NEC MSBU?"

print(generate_response(prompt))

Go to Tenant > Inquiries and reports > Tenant transaction table. Select the record to generate and click Electronic journal button. Click the OK button. The system will generate the electronic journal and will send to the specified recipient. The system will also send a notification to the user indicating that the electronic journal has been generated. The system will also send a notification to the recipient indicating that the electronic journal has been generated. The system will also send a notification to the user indicating that the electronic journal has been sent. The system will also send a notification to the recipient indicating that the electronic journal has been sent. The system will also send a notification to the user indicating that the electronic journal has been sent. The system will also send a notification to the recipient indicating that the electronic journal has been sent. The system will also send a notification to the user indicating that the electronic journa

***Observations***
* The Falcon 7b model has been trained on data upto the year 2020. So any new information has to be presented to the LLM and trained on it
* The first prompt shows that the Falcon 7b model even when fine tuned is able to retain and present coherent information, even on data it hasnt been trained on.
* The second prompt shows the response on the data which it has been trained on. Notice that the model has added to the reponse and to the data it has been trained on.
* The third prompt shows that it is able to retireve the current information based on the data it has been trianed on and able to add more relevant information to it.
* Problem in understanding acronymns like MSBU AD ID
* Unable to hold contextual information even after finetuning